# 11.1

- 모델을 초기화하고 데이터를 로딩한다.
- 어느정도 임의로 선택한 에포크 수로 루프를 반복한다.

# 11.2 애플리케이션의 메인 진입점

In [1]:
import datetime

from util import importstr
from logconf import logging
log = logging.getLogger('nb')

def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # 4코어 8스레드 CPU로 가정
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1)) 
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}).main()".format(app, argv))

training.py

In [2]:
import sys
import argparse

# __init__함수에서 표준 argparse라이브러리를 사용한다. 초기화 함수에 커스텀 인자를 전달한다.

class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--num-workers',
            help='Number of worker processes for background data loading',
            default=8,
            type=int,
        )
        parser.add_argument('--batch-size',
            help='Batch size to use for training',
            default=32,
            type=int,
        )
        parser.add_argument('--epochs',
            help='Number of epochs to train for',
            default=1,
            type=int,
        )

        

# 11.3
- 모델과 옵티마이저 초기화
- Dataset과 DataLoader인스턴스 초기화

In [ ]:
#!pip install SimpleITK
#!pip install diskcache

In [ ]:

import torch
from model import LunaModel
from torch.optim import SGD
import torch.nn as nn
from dsets import LunaDataset
from torch.utils.data import DataLoader

class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()
    ### 모델과 옵티마이저 초기화
    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1: #복수개의 GPU탐지
                model = nn.DataParallel(model) #모델 래핑
            model = model.to(self.device) #GPU에 모델 파라미터 전달
        return model
    
    def initOptimizer(self):
        return SGD(self.model.parameters(), lr=0.001, momentum=0.99)
        #하이퍼파라미터 탐색을 통해 다른 학습률과 모멘텀값으로 바꿀수 있음


    ### Dataset과 DataLoader인스턴스 초기화
    def initTrainDl(self):
        train_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=False,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader(
            train_ds,
            batch_size=batch_size, #알아서 배치로 나뉜다.
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda, #고정된 메모리 영역이 GPU쪽으로 빠르게 전송됨
        )

        return train_dl


    def initValDl(self):
        val_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=True, #train과 똑같이 하는데 여기만 다르게 해주면 됨.
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        val_dl = DataLoader(
            val_ds,
            batch_size=batch_size,
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,
        )

##나중에 메인함수에서 할당만 시키면 됨.
    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()

# 11.4
- 테일: 입력을 신경망에 넣기 전 처리과정을 담달하는 제일 첫 부분의 일부 계층
  - 백본이 원하는 형태로 입력을 만들어야 하므로 신경망의 나머지 부분과는 구조나 구성이 다른 경우가 많다. 테일에 컨볼루션 층이 들어간 경우는 이미지 크기를 공격적으로 다운 샘플링하기 위한 용도가 대부분이다.

- 백본: 여러계층을 가지는데 일반적으로는 연속된 블럭에 배치된다.
  - 각 블럭은 동일한 세트의 계층을 가지며 블럭과 블럭을 거칠 때마다 필요한 입력크기나 필터 수가 달라진다.
  - 여기에는 3x3컨볼루션 두개와 하나의 활성화 함수 그리고 블록 끝에 맥스풀링 연산이 이어진다.
- 헤드: 백본의 출력을 받아 원하는 출력 형태로 바꾼다. 컨볼루션 신경망에서 이 작업을 중간 출력물을 flatten하고 완전 연결계층에 전달하는 역할을 하기도 한다.

In [24]:
#블럭은 아래처럼
# (컨볼루션-> 활성화함수-> 컨볼루션-> 활성화함수-> 맥스풀링)인 블럭을 정의
class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)

In [26]:
import math
#전체모델
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        ##테일 부분, 이 모델은 이미지가 충분히 작아 다운샘플링없이 배치정규화만 구성.
        self.tail_batchnorm = nn.BatchNorm3d(1)


        #백본: 위에 정의한 블록을 4번 쌓아올림
        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)


        #헤드: 백본의 출력을 fc층과 연결시킨 후 softmax통과
        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)

        self._init_weights()
    
    #파라미터 초기화부분
    #1보다 작은 가중치는 잔차연결이 없는 경우 매우 좋지 않은 성능을 낼 수 있다. 때문에 가중치 초기화부분도 모델의 성능에 영향을 준다.
    def _init_weights(self):
        for m in self.modules():
            if type(m) in {
                nn.Linear,
                nn.Conv3d,
                nn.Conv2d,
                nn.ConvTranspose2d,
                nn.ConvTranspose3d,
            }:
                nn.init.kaiming_normal_(
                    m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
                )
                if m.bias is not None:
                    fan_in, fan_out = \
                        nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)



    def forward(self, input_batch):
        bn_output = self.tail_batchnorm(input_batch)

        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)

        #평탄화 작업을 해야함.
        conv_flat = block_out.view( 
            block_out.size(0), #배치크기
            -1,
        )
        linear_output = self.head_linear(conv_flat)

        #반환값은 로지트와 소프트맥스 확룰을 만든다. 로지트는 손실함수 계산을 위해 사용되고 소프트맥스 확률은 실제 분류시 확률값을 이용한다.
        return linear_output, self.head_softmax(linear_output)

# 11.5

In [27]:
# 모델 훈련

METRICS_SIZE = 3
from util import enumerateWithEstimate

class LunaTrainingApp:
  def main(self):

  #에포크마다 훈련 결과를 남기기
    for epoch_ndx in range(1, self.cli_args.epochs + 1):
      trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
      self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

  def doTraining(self, epoch_ndx, train_dl):
        self.model.train()

        #빈 메트릭 배열을 초기화
        trnMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(train_dl.dataset),
            device=self.device,
        )

        #시간을 예측하며 배치루프를 설정한다.
        batch_iter = enumerateWithEstimate(
            train_dl,
            "E{} Training".format(epoch_ndx),
            start_ndx=train_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.optimizer.zero_grad() #그라디언트 0으로 초기화


            #손실함수 계산
            loss_var = self.computeBatchLoss(
                batch_ndx,
                batch_tup,
                train_dl.batch_size,
                trnMetrics_g
            )

            loss_var.backward()

            #파라미터 업데이트
            self.optimizer.step()


        self.totalTrainingSamples_count += len(train_dl.dataset)

        return trnMetrics_g.to('cpu')

In [28]:
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2

class LunaTrainingApp:

  #훈련루프와 검증루프 둘다에서 호출될 함수이다. 샘플배치에 대한 손실을 계산하고, 각 샘플에 대해 모델이 만들어내는 출력 정보도 기록한다. 
  #이를 통해 각 클래스별로 계산이 얼마나 정확한지 백분율로 알아보고 분류가 잘 되지 않는 클래스를 찾아 집중개선 가능하다.
  def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
        input_t, label_t, _series_list, _center_list = batch_tup

        input_g = input_t.to(self.device, non_blocking=True) #GPU로 이동
        label_g = label_t.to(self.device, non_blocking=True) #GPU로 이동

        logits_g, probability_g = self.model(input_g) #모델의 출력값을 저장

        loss_func = nn.CrossEntropyLoss(reduction='none') #샘플별 손실값을 얻음
        loss_g = loss_func(logits_g,label_g[:,1]) #손실값: logits_g와 원핫인코딩 클래스의 인덱스인 label_g[:,1]사이의 로스

        #추후 분석을 위해 샘플별 통계값 기록
        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + label_t.size(0)

        metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = label_g[:,1].detach()
        metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = probability_g[:,1].detach()
        metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = loss_g.detach()

        return loss_g.mean()

In [29]:
# 검증루프
class LunaTrainingApp:
  def main(self):
        for epoch_ndx in range(1, self.cli_args.epochs + 1):

            valMetrics_t = self.doValidation(epoch_ndx, val_dl)
            self.logMetrics(epoch_ndx, 'val', valMetrics_t)

  def doValidation(self, epoch_ndx, val_dl):
        with torch.no_grad():
            self.model.eval() #훈련때 사용했던 기능은 끈다.
            valMetrics_g = torch.zeros(
                METRICS_SIZE,
                len(val_dl.dataset),
                device=self.device,
            )

            batch_iter = enumerateWithEstimate(
                val_dl,
                "E{} Validation ".format(epoch_ndx),
                start_ndx=val_dl.num_workers,
            )
            for batch_ndx, batch_tup in batch_iter:
                self.computeBatchLoss(
                    batch_ndx, batch_tup, val_dl.batch_size, valMetrics_g)

        return valMetrics_g.to('cpu')

# 11.6
각 에포크마다 진행사항은 trnMetrics_g와 valMetrics_g에 수집했다. 또한 텐서에는 훈련과 검증단계에 대한 클래스별로 백분율이나 평균손실값을 계산하기 위해 모든 데이터가 포함되어 있으므로 이를 통해 성능을 알아볼 수 있다.

In [35]:
import numpy as np
def logMetrics(
            self,
            epoch_ndx, #결과를 로깅할때 인덱스를 표시하는 용도
            mode_str, #메트릭이 훈련용인지 검증용인지 표시
            metrics_t, #결과를 담고있는 용도
            classificationThreshold=0.5):
  
  #마스크 구성: 결절샘플또는 비결절샘플에 대해서만 메트릭을 제한하는 마스크이다. 또한 클래스별 총 샘플 수와 잘 분류된 샘플 수를 알아볼 수 있다.
  negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold #실제값들 중 0.5보다 작은 것들에 대해서 불리언값으로 가지고있음
  negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold #예측값들 중 0.5보다 작은 것들에 대해서 불리언값으로 가지고있음

  posLabel_mask = ~negLabel_mask #실제값들 중 0.5보다 큰 것들에 대해서 불리언값으로 가지고있음
  posPred_mask = ~negPred_mask #예측값들 중 0.5보다 큰 것들에 대해서 불리언값으로 가지고있음

  neg_count = int(negLabel_mask.sum()) #실제값들중 음성인 개수
  pos_count = int(posLabel_mask.sum()) #실제값들중 양성인 개수

  neg_correct = int((negLabel_mask & negPred_mask).sum()) #음성으로 맞게 분류한 것들의 개수
  pos_correct = int((posLabel_mask & posPred_mask).sum()) #양성으로 맞게 분류한 것들의 개수

  #계산한 값들 전부 딕셔너리에 저장
  metrics_dict = {}
  metrics_dict['loss/all'] = metrics_t[METRICS_LOSS_NDX].mean()
  metrics_dict['loss/neg'] = metrics_t[METRICS_LOSS_NDX, negLabel_mask].mean()
  metrics_dict['loss/pos'] = metrics_t[METRICS_LOSS_NDX, posLabel_mask].mean()

  metrics_dict['correct/all'] = (pos_correct + neg_correct)/np.float32(metrics_t.shape[1]) * 100
  metrics_dict['correct/neg'] = neg_correct / np.float32(neg_count) * 100
  metrics_dict['correct/pos'] = pos_correct / np.float32(pos_count) * 100

  #저장한 결과 로깅
  log.info(
            ("E{} {:8} {loss/all:.4f} loss, "
                 + "{correct/all:-5.1f}% correct, "
            ).format(
                epoch_ndx,
                mode_str,
                **metrics_dict,
            )
        )
  log.info(
            ("E{} {:8} {loss/neg:.4f} loss, "
                 + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_neg',
                neg_correct=neg_correct,
                neg_count=neg_count,
                **metrics_dict,
            )
        )
  log.info(
            ("E{} {:8} {loss/pos:.4f} loss, "
                 + "{correct/pos:-5.1f}% correct ({pos_correct:} of {pos_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_pos',
                pos_correct=pos_correct,
                pos_count=pos_count,
                **metrics_dict,
            )
        )
  


# 11.7


In [38]:
import time
import random
for i,_ in enumerateWithEstimate(list(range(234)),"training"):
  time.sleep(random.random())
#해당 함수를 이용해 프로젝트의 훈련시간이 어느정도 걸리는지 예측 가능하다.

2022-09-16 06:51:21,421 WARNING  pid:57 util:209:enumerateWithEstimate training ----/234, starting
2022-09-16 06:51:23,642 INFO     pid:57 util:229:enumerateWithEstimate training    4/234, done at 2022-09-16 06:53:04, 0:01:43
2022-09-16 06:51:28,922 INFO     pid:57 util:229:enumerateWithEstimate training   16/234, done at 2022-09-16 06:53:04, 0:01:43
2022-09-16 06:51:52,897 INFO     pid:57 util:229:enumerateWithEstimate training   64/234, done at 2022-09-16 06:53:14, 0:01:53
2022-09-16 06:53:19,785 WARNING  pid:57 util:240:enumerateWithEstimate training ----/234, done at 2022-09-16 06:53:19
